<a href="https://colab.research.google.com/github/Marcosrsfilho10/Ola-mundo/blob/main/Base_de_chamados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install gspread pandas google-auth requests

In [ ]:
from google.colab import auth
auth.authenticate_user()

print("OK: autenticado no Google")

OK: autenticado no Google


In [ ]:
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

print("OK: gspread conectado")

OK: gspread conectado


In [ ]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1Y93cbA2_uUqhCNDJmPokopt5c8OwUTIo13XMiNAF4SU/edit?gid=1421018968#gid=1421018968"
sh = gc.open_by_url(SHEET_URL)

print("Abriu a planilha:", sh.title)
print("Abas:", [w.title for w in sh.worksheets()])

Abriu a planilha: Chamados Tecnicos - 2026
Abas: ['Base_Chamados', 'Chamados_Por_Mes', 'Chamados_por_Cliente', 'Chamados_por_Tecnico', 'Chamados_Abertos']


In [ ]:
import pandas as pd

# 1) Abrir a aba Base_Chamados
ws = sh.worksheet("Base_Chamados")

# 2) Criar um mini dataframe de teste
df_teste = pd.DataFrame([
    {"Id": 1, "Cliente Nome": "CLIENTE TESTE", "Data Abertura": "2026-01-29", "Situacao": "ABERTO"}
])

# 3) Limpar e escrever cabeçalho + dados
ws.clear()
ws.update([df_teste.columns.tolist()] + df_teste.values.tolist())

print("OK: escrevi um teste em Base_Chamados")

OK: escrevi um teste em Base_Chamados


In [ ]:
import requests

cookie = "ASP.NET_SessionId=31fmuaflpexvx5q4bjgmls4x; osVisitor=c6f823b4-636d-475f-9067-124033ea3123; DEVICE_SIMULATION=true; EPATaskbox.3320=0||NaN|NaN; osVisit=6443ef06-aff2-432f-93f7-4321faedd374; Users.sid=537313233496869626310865683496881256021; Users=98640151-c3ef-433e-b3b8-6adc14fc8b15; DEVICE_TYPE=tablet; pageLoadedFromBrowserCache=true; RT=s=1769777586832&r=https%3A%2F%2Fapp.liftflex.com%2FLiftflex_Relatorios%2FChamadosTecnicos.aspx"

url = "https://app.liftflex.com/EPA_Taskbox/API_GetActivityCount.aspx?UserId=3320"

headers = {
    "Cookie": cookie,
    "User-Agent": "Mozilla/5.0"
}

r = requests.get(url, headers=headers, timeout=30)

print("Status:", r.status_code)
print("Content-Type:", r.headers.get("content-type"))
print("Primeiros 300 chars:")
print(r.text[:300])

Status: 200
Content-Type: text/plain
Primeiros 300 chars:
{ "Activities" : 0, "HasUnseen" : false }


In [ ]:
import requests
from datetime import date
from urllib.parse import parse_qs, urlencode

URL = "https://app.liftflex.com/Liftflex_Relatorios/ChamadosTecnicos.aspx"

COOKIE = (
    "ASP.NET_SessionId=31fmuaflpexvx5q4bjgmls4x; "
    "osVisitor=c6f823b4-636d-475f-9067-124033ea3123; "
    "DEVICE_SIMULATION=true; "
    "EPATaskbox.3320=0||NaN|NaN; "
    "osVisit=6443ef06-aff2-432f-93f7-4321faedd374; "
    "Users.sid=537313233496869626310865683496881256021; "
    "Users=98640151-c3ef-433e-b3b8-6adc14fc8b15; "
    "DEVICE_TYPE=tablet; "
    "pageLoadedFromBrowserCache=true; "
    "RT=s=1769777586832&r=https%3A%2F%2Fapp.liftflex.com%2FLiftflex_Relatorios%2FChamadosTecnicos.aspx"
)

PAYLOAD_BASE = """__EVENTTARGET=DublinThemeOriginal_wt3%24block%24wtMainContent%24wt71&__EVENTARGUMENT=&__OSVSTATE=GDftbgSmfB7X%2F66fIq3V4bkcoHRj9zu8%2BOARL8wSphE0%2B%2B6%2B3G9Syf3BR9DDLLM3QiSWz%2BalBX9OC%2BhqMBZzDYfLgdsXAdEcZTAR%2B09yeBqkzND7zaqvmfs%2BJvyA7BZMMB7Z9n4T5%2Bhha%2F4jBEdsceFRlL5WuyBkkJ7E4dss2fIzcvpjzZ6JQXcnjeEetfoeSHBTvwJJK0TFKjwnGVkgiustCzw1k%2FjyFZLe8T8oY2o1%2BMHz257UAzyVpvvURz7ZguPcbjX10CuFKPcZ%2BGSx10AlM62bz3ncK%2BfxRpwh1ar1m8pJKn1TvqkV3fIAhMD0tOICV6os2mVASXwA%2FArToQlXXZrqbNyvQetQx%2FzImCFxSP8ZytChZZGpgmCT94tPSc8jvu%2FDyKnT%2B1ZbrN2%2BCV5lZlNanHKTUVhVMEnD%2BCjsmfir0VvRxhsGObthZP6gO5fAFZo0rBIjP4ty6EfwKwL3%2BxFl19GQjFVTuFQ1JVyiyg6qNuOl%2FqKsfrgznlvlsTnT9A8N7WnRxHnITa%2FTzY4osLnxYdcOVqpOwaCvEFX9I50K%2BjIw7jOulXASmyZk6QGV1%2FFNh9WRQHIqMuvxo%2FvVsCGsRcL66IGnCdWw%2FbYAA%2BlL0WUFo%2BBihSVAD9iz3zn6nay89yhmInEMcEkkA57Mr%2FSoiRAUcDVaRR%2F%2BK74VKlsk1klh1W%2B8kK9WJfA4N3Y0iSON5b1ATW9pbWtdSxke5iMCZX76XDrK2tIP4Ep8h45JpL95EUQ9wgVSCbCBD7Wr%2FOIyQC4gqWPtpgsE3vtNcdXuri%2Ffia%2F%2B%2FMfxReXdoR5TqrjCb4eMWRkoMX%2FX0GfQ4QSokdllkpnJs1KDXTslkYHEaGv4yCTrp9Y9xpNdnvLeHZZf7xumOUnR3%2FcN6qGxZE1MkMHpilTytlQts%2F8sj30TFBSgbZINO%2BOOxetg31d%2BJXCSVW%2FLgzX8gXDeNQYNyHbmfQBKhxel5MV5eOaU2uEEq8XVaIp%2FCeL4v0PFlIrOziFRNU4%2BF0YD%2Fqn9JPK%2BHn%2FBoxJHFimMLTcV0Lii8M9M75Y6yJCEO7QSpbDA8mZ0IK%2FF203X2Y8MGcaMbz6MdErkybwP7FC%2Bcj8H98HNEFMAuy9BQ%2FsnTPLyy4Prfhw9Eid5OWF0OVYAFpec%2FWxfnHp9l49SFcXizb33fgwIZkw4cSZHe5JbAk4SNG%2BQfyryCPIKyMni2rnQVvWP8bt3g7%2Fw0%2BpV2DmUBdvSBOhBniCyY46M%2BjMGTcV6PdNTQIQX%2BXWs%2BZ0xoxB2FyBFrvE08tsZRbQw5xqX1Vg1%2FBQe6vj%2B4D1dnAxh3UoFjkUC9ueaG8jes9nj1DKb9aSFTNhFVld6ak0YKp2qYHWGL43Omn7ywU3gc7KK1AOH2K4%3D&__VIEWSTATE=&__VIEWSTATEGENERATOR=3F270145&DublinThemeOriginal_wt3%24block%24wtHeader%24Liftflex_Layout_wt30%24block%24wtComboEmpresas=778&DublinThemeOriginal_wt3%24block%24wtMainContent%24wtInputSituacao=__ossli_0&DublinThemeOriginal_wt3%24block%24wtMainContent%24wt99=&DublinThemeOriginal_wt3%24block%24wtMainContent%24wt90=&DublinThemeOriginal_wt3%24block%24wtMainContent%24wt13%24autocomplete_wt1%24block%24wtSearchInputWidget%24wtInputSearch=&DublinThemeOriginal_wt3%24block%24wtMainContent%24Liftflex_WebBlocks_wt7%24block%24autocomplete_wt3%24block%24wtSearchInputWidget%24wtInputSearch=&DublinThemeOriginal_wt3%24block%24wtMainContent%24wt64=__ossli_0&DublinThemeOriginal_wt3%24block%24wtMainContent%24wtInput_ContratoNome=&DublinThemeOriginal_wt3%24block%24wtMainContent%24RichWidgets_wt36%24block%24wttxtAutoCompleteValue=--unchanged--&DublinThemeOriginal_wt3%24block%24wtMainContent%24RichWidgets_wt36%24block%24wttxtAutoCompleteAux=isVisible&DublinThemeOriginal_wt3%24block%24wtMainContent%24wtInput_PredioNome=&DublinThemeOriginal_wt3%24block%24wtMainContent%24RichWidgets_wt45%24block%24wttxtAutoCompleteValue=--unchanged--&DublinThemeOriginal_wt3%24block%24wtMainContent%24RichWidgets_wt45%24block%24wttxtAutoCompleteAux=isVisible&DublinThemeOriginal_wt3%24block%24wtMainContent%24wtInput_ElevadorNome=&DublinThemeOriginal_wt3%24block%24wtMainContent%24RichWidgets_wt119%24block%24wttxtAutoCompleteValue=--unchanged--&DublinThemeOriginal_wt3%24block%24wtMainContent%24RichWidgets_wt119%24block%24wttxtAutoCompleteAux=isVisible&DublinThemeOriginal_wt3%24block%24wtMainContent%24wt102=__ossli_0&DublinThemeOriginal_wt3%24block%24wtMainContent%24wt117=2&DublinThemeOriginal_wt3%24block%24wtMainContent%24wtInput_DataInicial=01%2F01%2F2026&DublinThemeOriginal_wt3%24block%24wtMainContent%24wtInput_DataFinal=30%2F01%2F2026&DublinThemeOriginal_wt3%24block%24wtMainContent%24wtInputTipoRelatorio=1&DublinThemeOriginal_wt3%24block%24wtMainContent%24wt112=1&DublinThemeOriginal_wt3%24block%24WebPatterns_wt10%24block%24wt4%24wt43%24wtCheckbox%24wt37=on"""

headers = {
    "Cookie": COOKIE,
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "User-Agent": "Mozilla/5.0",
    "Origin": "https://app.liftflex.com",
    "Referer": URL,
}

# ----------------------------------------------------
# 1) Converte payload string -> dict
# ----------------------------------------------------
payload_dict = parse_qs(PAYLOAD_BASE, keep_blank_values=True)

# ----------------------------------------------------
# 2) Define intervalo: 01/01/2026 até HOJE
# ----------------------------------------------------
payload_dict[
    "DublinThemeOriginal_wt3$block$wtMainContent$wtInput_DataInicial"
] = ["01/01/2026"]

payload_dict[
    "DublinThemeOriginal_wt3$block$wtMainContent$wtInput_DataFinal"
] = [date.today().strftime("%d/%m/%Y")]

# ----------------------------------------------------
# 3) Re-encoda corretamente (URL-encoded)
# ----------------------------------------------------
payload = urlencode(payload_dict, doseq=True)

# ----------------------------------------------------
# 4) POST
# ----------------------------------------------------
s = requests.Session()
r = s.post(URL, data=payload, headers=headers, timeout=60, allow_redirects=True)

print("Status:", r.status_code)
print("Content-Type:", r.headers.get("Content-Type"))
print("Content-Disposition:", r.headers.get("Content-Disposition"))

# ----------------------------------------------------
# 5) Salva XLSX
# ----------------------------------------------------
fname = "ChamadosTecnicos.xlsx"

if (
    ".xlsx" in (r.headers.get("Content-Disposition") or "").lower()
    or "spreadsheet" in (r.headers.get("Content-Type") or "").lower()
    or r.content[:2] == b"PK"
):
    with open(fname, "wb") as f:
        f.write(r.content)
    print("✅ Excel gerado:", fname, "tamanho:", len(r.content))
else:
    print("❌ Liftflex retornou HTML (cookie expirado ou filtro inválido)")
    print(r.text[:800])


Status: 200
Content-Type: application/octet-stream
Content-Disposition: attachment; filename="Chamados Tecnicos.xlsx"
✅ Excel gerado: ChamadosTecnicos.xlsx tamanho: 18403


In [ ]:
import pandas as pd

arquivo = "ChamadosTecnicos.xlsx"  # tem que ser o mesmo nome que você salvou no download
df = pd.read_excel(arquivo)

print("Linhas:", df.shape[0])
print("Colunas:", df.shape[1])
display(df.head(10))

Linhas: 98
Colunas: 11


,Id,Cliente Nome,Data Fechamento,Data Abertura,Predio Nome,Elevador Nome,Numero,Situacao,Descricao,Observacao,Tecnico Nome
0,1909215,CONDOMINIO DO EDIFICIO PINHEIROS,2026-01-01 10:31:09,2026-01-01 09:19:56,CONDOMINIO DO EDIFICIO PINHEIROS,ELEVADOR SOCIAL 2,X,Finalizado,Elevador com botoeira quebrada,Feito colagem do visor do P social 02,ALLYSON VINICIUS SOARES DOS SANTOS
1,1909290,CONDOMINIO DO EDIFICIO RIVIERA,2026-01-01 15:40:42,2026-01-01 14:20:56,EDIFICIO RIVIERA,ELEVADOR DE SERVIÇO - BLOCO B,X,Finalizado,Elevador de serviço \nBloco B\nParado,Feito regulagem do trinco do térreo\n,ALLYSON VINICIUS SOARES DOS SANTOS
2,1909336,CONDOMINIO DO EDIFICIO LOTUS,2026-01-01 19:06:55,2026-01-01 17:13:40,CONDOMINIO DO EDIFICIO LOTUS,ELEVADOR SOCIAL,X,Finalizado,Elevador não abre porta externa,Feito reset no comando.,ALLYSON VINICIUS SOARES DOS SANTOS
3,1909412,CONDOMINIO DO EDIFICIO UBAIAS PRINCE,2026-01-02 11:47:25,2026-01-02 06:37:48,CONDOMINIO DO EDIFICIO UBAIAS PRINCE,ELEVADOR SOCIAL 2,X,Finalizado,Elevador social 2 \nBotão de cabine quebrado,Ajustado mola de acionamento de emergência da ...,LAECIO SILVA PALMERA
4,1909413,CONDOMINIO DO EDIFICIO SANTO INACIO,2026-01-02 08:43:13,2026-01-02 06:39:27,CONDOMINIO DO EDIFICIO SANTO INACIO,ELEVADOR SOCIAL 2,X,Finalizado,Elevador social 2\nParado,Funcionando normalmente. Feito revisões nos co...,LAECIO SILVA PALMERA
5,1909415,CONDOMINIO DO EDIFICIO ENSEADA DA BARRA,2026-01-02 09:49:04,2026-01-02 06:55:20,ENSEADA DA BARRA,ELEVADOR DE SERVIÇO,X,Finalizado,Elevador de serviço \nabrindo e fechando porta...,Ajustado contato de trinco e PC e liberado o e...,LAECIO SILVA PALMERA
6,1909448,CONDOMINIO DO EDIFICIO RIVIERA,2026-01-02 13:00:00,2026-01-02 08:05:05,EDIFICIO RIVIERA,ELEVADOR SOCIAL - BLOCO B,X,Finalizado,Elevador social bloco B\nPorta externa não abre,Estacionamento de porta fechada cancelado.\nFe...,ALLYSON VINICIUS SOARES DOS SANTOS
7,1909608,CONDOMINIO DO EDIFICIO PRAIA DE CALHETAS,2026-01-02 11:05:55,2026-01-02 10:14:02,CONDOMINIO DO EDIFICIO PRAIA DE CALHETAS,ELEVADOR SOCIAL,X,Finalizado,Elevador social \nPorta Caiu,Realizado limpeza de cola velha e colado botoe...,LAECIO SILVA PALMERA
8,1909819,EDIFICIO LADY ROBERTA,2026-01-02 17:38:22,2026-01-02 13:29:59,EDIFICIO LADY ROBERTA,ELEVADOR SOCIAL,X,Finalizado,Elevador social \nAbrindo e fechando \nPorta!,Ajustado sensores da porta de cabina ( BPE ) ✅️,LAECIO SILVA PALMERA
9,1909849,EDIFICIO MATRIZ CENTER,NaT,2026-01-02 13:50:27,EDIFICIO MATRIZ CENTER,ELEVADOR SOCIAL,X,Aberto,Gente presa,NaN,JOSÉ LUCAS NUNES BERNARDO


In [ ]:
!pip install gspread gspread_dataframe google-auth

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default

SPREADSHEET_NAME = "Chamados Tecnicos - 2026"
ABA_BASE = "Base_Chamados"
ARQUIVO_XLSX = "ChamadosTecnicos.xlsx"

df = pd.read_excel(ARQUIVO_XLSX)
df.columns = df.columns.str.strip()

creds, _ = default()
gc = gspread.authorize(creds)

sh = gc.open(SPREADSHEET_NAME)
ws = sh.worksheet(ABA_BASE)

ws.batch_clear([f"A2:Z{ws.row_count}"])

set_with_dataframe(
    ws,
    df,
    row=2,
    col=1,
    include_column_header=False,
    resize=False
)

print("✅ Base_Chamados atualizada com sucesso")

✅ Base_Chamados atualizada com sucesso
